In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

#from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.pgmpy_BN import Pgmpy_BN

In [2]:
with open('clt.pkl', 'rb') as f:
    BN = pickle.load(f)

In [53]:
import itertools
import networkx as nx
import numpy as np
import time
try:
    from tqdm import tqdm
except:
    import tqdm
from collections import defaultdict
from itertools import chain
import copy
from Pgmpy.inference import Inference
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD


class VariableEliminationJIT(object):
    def __init__(self, model, cpds, topological_order, topological_order_node, probs=None, root=True):
        model.check_model()
        self.cpds = cpds
        self.topological_order = topological_order
        self.topological_order_node = topological_order_node
        self.model = model
        if probs is not None:
            self.probs = probs
        else:
            self.probs = dict()


        self.variables = model.nodes()

        if root:
            self.root = self.get_root()

    def get_root(self):
        """Returns the network's root node."""

        def find_root(graph, node):
            predecessor = next(self.model.predecessors(node), None)
            if predecessor:
                root = find_root(graph, predecessor)
            else:
                root = node
            return root

        return find_root(self, list(self.model.nodes)[0])

    def steiner_tree(self, nodes):
        """Returns the minimal part of the tree that contains a set of nodes."""
        sub_nodes = set()

        def walk(node, path):
            if len(nodes) == 0:
                return

            if node in nodes:
                sub_nodes.update(path + [node])
                nodes.remove(node)

            for child in self.model.successors(node):
                walk(child, path + [node])

        walk(self.root, [])
        sub_graph = self.model.subgraph(sub_nodes)
        sub_graph.cardinalities = defaultdict(int)
        for node in sub_graph.nodes:
            sub_graph.cardinalities[node] = self.model.cardinalities[node]
        return sub_graph


    def _get_working_factors(self, query=None, fanout=[]):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """
        useful_var = list(query.keys()) + fanout
        sub_graph_model = self.steiner_tree(useful_var)

        elimination_order = []
        working_cpds = []
        working_factors = dict()
        for i, node in enumerate(self.topological_order_node[::-1]):
            ind = len(self.topological_order_node)-i-1
            if node in sub_graph_model.nodes:
                elimination_order.append(node)
                cpd = copy.deepcopy(self.cpds[ind])
                working_cpds.append(cpd)
                working_factors[node] = [cpd]

        for node in sub_graph_model.nodes:
            for cpd in working_cpds:
                if node != cpd.variable and node in cpd.variables:
                    working_factors[node].append(cpd)
        
        return working_factors, sub_graph_model, elimination_order


    def query(self, query, n_distinct=None):
        """
        Compiles a ppl program into a fixed linear algebra program to speed up the inference
        ----------
        query: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence
        n_distinct: dict
            a dict key, value pair as {var: probability of observed value in state}
            This is for the case, where we bin the continuous or large domain so each state now contains many observed
            value. Default to none, meaning no large domain.
        """
        working_factors, sub_graph_model, elimination_order = self._get_working_factors(query)
        for i, var in enumerate(elimination_order):
            root_var = i == (len(elimination_order) - 1)
            #print(var, len(working_factors[var]), root_var)
            if len(working_factors[var]) == 1:
                #leaf node in BN
                if var in query:
                    #print(var, query[var], n_distinct[var])
                    new_value = working_factors[var][0].values
                    if n_distinct:
                        if len(n_distinct[var]) == 1:
                            new_value = new_value[query[var]] * n_distinct[var][0]
                        else:
                            new_value = np.dot(n_distinct[var], new_value[query[var]])
                    else:
                        new_value = np.sum(new_value[query[var]], axis=0)
                    new_value = new_value.reshape(-1)
                    if root_var:
                        return new_value
                    assert len(new_value.shape) == 1, \
                        f"unreduced variable {working_factors[var][0].variables}, {new_value} with shape {new_value.shape}" 
                else:
                    if root_var:
                        return 1
                    new_value = np.ones(working_factors[var][0].values.shape[-1])
                
                assert len(new_value.shape) == 1, f"unreduced variable {var}"
                working_factors[var][0].values = new_value
            else:
                if var in query:
                    if type(query[var]) != list:
                        assert type(query[var]) == int, f"invalid query {query[var]}"
                        query[var] = [query[var]]
                    self_value = working_factors[var][0].values[query[var]]  #Pr(var|Parent(var))
                    if n_distinct:
                        self_value = (self_value.transpose() * n_distinct[var]).transpose()
                    children_value = []
                    #check if all children has been reduced
                    for cpd in working_factors[var][1:]:
                        #print("y")
                        #print(cpd.variables)
                        child_value = cpd.values[query[var]]    #M(var) = Pr(child(var)|var)
                        assert len(child_value.shape) == 1, \
                        f"unreduced children {cpd.variables}, {child_value} with shape {child_value.shape}"
                        children_value.append(child_value)
                    if len(children_value) == 1:
                        children_value = children_value[0]
                    else:
                        #print(children_value)
                        children_value = np.prod(np.stack(children_value), axis=0)
                    if root_var:
                        new_value = np.dot(self_value, children_value)
                        return new_value
                    new_value = np.dot(np.transpose(self_value), children_value)
                else:
                    self_value = working_factors[var][0].values  # Pr(var|Parent(var))
                    children_value = []
                    # check if all children has been reduced
                    for cpd in working_factors[var][1:]:
                        child_value = cpd.values  # M(var) = Pr(child(var)|var)
                        assert len(child_value.shape) == 1, \
                        f"unreduced children {cpd.variables}, {child_value} with shape {child_value.shape}"
                        children_value.append(child_value)
                    if len(children_value) == 1:
                        children_value = children_value[0]
                    else:
                        children_value = np.prod(np.stack(children_value), axis=0)
                    if root_var:
                        new_value = np.dot(self_value, children_value)
                        return new_value
                    new_value = np.dot(np.transpose(self_value), children_value)
                assert len(new_value.shape) == 1, f"unreduced variable {var}"
                working_factors[var][0].values = new_value
        return 0

    def expectation(self, query, fanout_attrs, fanout_values, n_distinct=None):
        """
        Compiles a ppl program into a fixed linear algebra program to speed up the expectation inference
        """
        working_factors, sub_graph_model, elimination_order = self._get_working_factors(query, fanout_attrs)
        for i, var in enumerate(elimination_order):
            root_var = i == (len(elimination_order) - 1)
            #print(var, len(working_factors[var]))
            if len(working_factors[var]) == 1:
                #leaf node in BN
                if var in query:
                    new_value = working_factors[var][0].values
                    #print(new_value.shape)
                    if n_distinct:
                        #print(var, query[var], n_distinct[var])
                        if len(n_distinct[var]) == 1:
                            new_value = new_value[query[var]] * n_distinct[var][0]
                            new_value = new_value.reshape(-1)
                        else:
                            new_value = np.dot(n_distinct[var], new_value[query[var]])
                    else:
                        new_value = np.sum(new_value[query[var]], axis=0)
                    new_value = new_value.reshape(-1)
                    if root_var:
                        return new_value
                elif var in fanout_attrs:
                    assert not root_var, "no querying variables"
                    new_value = working_factors[var][0].values
                    #print(new_value.shape)
                    new_value = np.dot(fanout_values[var], new_value)
                else:
                    if root_var:
                        return 1
                    new_value = np.ones(working_factors[var][0].values.shape[-1])
                    
                #print(new_value)
                assert len(new_value.shape) == 1, f"unreduced variable {var} with shape {new_value.shape}"
                working_factors[var][0].values = new_value
            else:
                if var in query:
                    if type(query[var]) != list:
                        assert type(query[var]) == int, f"invalid query {query[var]}"
                        query[var] = [query[var]]
                    self_value = working_factors[var][0].values[query[var]]  #Pr(var|Parent(var))
                    if n_distinct:
                        self_value = (self_value.transpose() * n_distinct[var]).transpose()
                    children_value = []
                    #check if all children has been reduced
                    for cpd in working_factors[var][1:]:
                        #print("y")
                        #print(cpd.variables)
                        child_value = cpd.values[query[var]]    #M(var) = Pr(child(var)|var)
                        assert len(child_value.shape) == 1, \
                        f"unreduced children {cpd.variables}, {child_value} with shape {child_value.shape}"
                        children_value.append(child_value)
                    if len(children_value) == 1:
                        children_value = children_value[0]
                    else:
                        #print(children_value)
                        children_value = np.prod(np.stack(children_value), axis=0)
                    if root_var:
                        new_value = np.dot(self_value, children_value)
                        return new_value
                    new_value = np.dot(np.transpose(self_value), children_value)

                else:
                    self_value = working_factors[var][0].values  # Pr(var|Parent(var))
                    if var in fanout_attrs:
                        self_value = (self_value.transpose() * fanout_values[var]).transpose()
                    children_value = []
                    # check if all children has been reduced
                    for cpd in working_factors[var][1:]:
                        #print(cpd.variables)
                        child_value = cpd.values  # M(var) = Pr(child(var)|var)
                        assert len(child_value.shape) == 1, \
                        f"unreduced children {cpd.variables}, {child_value} with shape {child_value.shape}"
                        children_value.append(child_value)
                    if len(children_value) == 1:
                        children_value = children_value[0]
                    else:
                        children_value = np.prod(np.stack(children_value), axis=0)
                    if root_var:
                        new_value = np.dot(self_value, children_value)
                        return new_value
                    new_value = np.dot(np.transpose(self_value), children_value)
                assert len(new_value.shape) == 1, f"unreduced variable {var}"
                #print(new_value)
                working_factors[var][0].values = new_value
        return 0

In [54]:
def align_cpds_in_topological(BN):
    cpds = BN.model.cpds
    sampling_order = []
    while len(sampling_order) < len(BN.structure):
        for i, deps in enumerate(BN.structure):
            if i in sampling_order:
                continue  # already ordered
            if all(d in sampling_order for d in deps):
                sampling_order.append(i)
    topological_order = sampling_order
    topological_order_node = [BN.node_names[i] for i in sampling_order]
    new_cpds = []
    for n in topological_order_node:
        for cpd in cpds:
            if cpd.variable == n:
                new_cpds.append(cpd)
                break
    assert len(cpds) == len(new_cpds)
    return new_cpds, topological_order, topological_order_node

In [55]:
with open('/home/ziniu.wzn/BN_checkpoints/check_points/Census_chow-liu.pkl', 'rb') as f:
    BN = pickle.load(f)
BN.init_inference_method()

In [56]:
cpds, topological_order, topological_order_node = align_cpds_in_topological(BN)
ve = VariableEliminationJIT(BN.model, cpds, topological_order, topological_order_node)

In [69]:
tic = time.time()
for var in BN.fanouts:
    n_distinct[var] = torch.tensor(copy.deepcopy(BN.fanouts[var]), dtype=torch.float64).cuda()
print(time.time()-tic)


0.00215911865234375


In [57]:
topological_order_node

['dAge',
 'dIncome5',
 'dIncome7',
 'iRrelchld',
 'iRspouse',
 'iSchool',
 'iYearsch',
 'iYearwrk',
 'iDisabl2',
 'iLang1',
 'iMarital',
 'iMobility',
 'dOccup',
 'iRelat1',
 'iRelat2',
 'iRemplpar',
 'iRlabor',
 'iRownchld',
 'iSubfam1',
 'iSubfam2',
 'iTmpabsnt',
 'iWork89',
 'iWorklwk',
 'dAncstry1',
 'dAncstry2',
 'iDisabl1',
 'iEnglish',
 'iFertil',
 'dHispanic',
 'dIndustry',
 'iLooking',
 'iMeans',
 'iMilitary',
 'iMobillim',
 'iPerscare',
 'dPOB',
 'dPoverty',
 'iRagechld',
 'iRiders',
 'iRPOB',
 'iRvetserv',
 'iSept80',
 'iSex',
 'dTravtime',
 'iVietnam',
 'dWeek89',
 'iWWII',
 'dYrsserv',
 'iAvail',
 'iCitizen',
 'iClass',
 'dDepart',
 'iFeb55',
 'dHours',
 'iImmigr',
 'dIncome2',
 'dIncome3',
 'iKorean',
 'iMay75880',
 'iOthrserv',
 'dPwgt1',
 'dRearning',
 'dRpincome',
 'dHour89',
 'dIncome1',
 'dIncome4',
 'dIncome6',
 'dIncome8']

In [58]:
BN.init_inference_method()
#with open("../Benchmark/DMV/query.sql") as f:
 #   queries = f.readlines()
with open("/home/ziniu.wzn/Census/cardinality/query_one_side.sql") as f:
    queries = f.readlines()
from Evaluation.cardinality_estimation import parse_query_single_table

In [63]:
import copy
query_str = queries[147].split("||")[0]
query_str = parse_query_single_table(query_str.strip(), BN)
print(query_str)
print(BN.query(copy.deepcopy(query_str)))
query_str, n_distinct = BN.query_decoding(query_str, None)
print(query_str)
tic = time.time()
a = ve.query(query_str, n_distinct)
print(time.time()-tic)
print(a*BN.nrows)
print(n_distinct)

{'iRelat2': [0], 'dTravtime': [1, 2, 0, 3], 'iYearsch': [11, 5, 10, 4, 8, 1, 7, 6, 2, 9, 3]}
1490293
{'iRelat2': [0], 'dTravtime': [3, 5, 0, 4], 'iYearsch': [1, 2, 0, 4, 7, 8, 6, 9, 15, 11, 14]}
(13,)
(14,)
(8,)
0.0021071434020996094
1490292.8364073704
{'iRelat2': [1], 'dTravtime': [1, 1, 1, 1], 'iYearsch': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [60]:
#with open("../Benchmark/DMV/query.sql") as f:
 #   queries = f.readlines()
with open("/home/ziniu.wzn/Census/cardinality/query_one_side.sql") as f:
    queries = f.readlines()

res = []
cards = []
lat = []
for q in queries:
    query_str = q.split("||")[0]
    card = int(q.split("||")[-1])
    cards.append(card)
    query_str = parse_query_single_table(query_str.strip(), BN)
    #print(BN.query(query_str))
    query_str, n_distinct = BN.query_decoding(query_str, None)
    tic=time.time()
    a = ve.query(query_str)
    print(time.time()-tic)
    lat.append(time.time()-tic)
    res.append(a*BN.nrows)

(12,)
(12,)
(13,)
(3,)
(14,)
(8,)
0.004031181335449219
(12,)
(12,)
(5,)
(12,)
(16,)
(8,)
0.004279613494873047
(3,)
(14,)
(3,)
(3,)
(3,)
(7,)
0.00347137451171875
(3,)
(10,)
(5,)
(7,)
(8,)
0.003063201904296875
(6,)
(6,)
(12,)
(10,)
(14,)
(12,)
0.0064792633056640625
(12,)
(13,)
(3,)
(8,)
(2,)
0.002795696258544922
(6,)
(12,)
(14,)
(12,)
(13,)
(14,)
(9,)
(2,)
(8,)
0.00519251823425293
(6,)
(12,)
(3,)
0.002370119094848633
(6,)
(12,)
(3,)
0.0022330284118652344
(12,)
(14,)
(12,)
(7,)
(16,)
(14,)
0.003599405288696289
(13,)
(12,)
(7,)
(9,)
(14,)
(14,)
(3,)
(8,)
0.004147529602050781
(13,)
(5,)
(14,)
(16,)
(8,)
0.0023157596588134766
(12,)
(14,)
(13,)
(12,)
(7,)
(2,)
(14,)
(8,)
0.004041433334350586
(12,)
(3,)
(3,)
0.0018846988677978516
(3,)
(12,)
(14,)
(12,)
0.003321409225463867
(6,)
(6,)
(13,)
(14,)
(3,)
0.0031108856201171875
(12,)
(12,)
(12,)
(3,)
(12,)
(14,)
(8,)
0.0026688575744628906
(6,)
(14,)
(14,)
(8,)
(8,)
0.0034933090209960938
(6,)
(6,)
(10,)
(12,)
(7,)
(4,)
0.004077911376953125
(13,)
(3,)


(6,)
(10,)
(12,)
(12,)
(2,)
0.003765583038330078
(6,)
(12,)
(14,)
(3,)
(3,)
0.0030677318572998047
(12,)
(3,)
(7,)
(14,)
(8,)
(8,)
(8,)
(8,)
0.003717660903930664
(10,)
(12,)
(3,)
(14,)
(12,)
(7,)
(14,)
0.004777193069458008
(14,)
(3,)
(3,)
(3,)
(12,)
0.002937793731689453
(6,)
(12,)
(14,)
0.002454519271850586
(6,)
(3,)
(3,)
(7,)
(14,)
(8,)
0.0033006668090820312
(6,)
(12,)
(10,)
(3,)
(7,)
0.0038406848907470703
(12,)
(10,)
(13,)
(13,)
(7,)
0.002852916717529297
(13,)
(3,)
(12,)
(12,)
(7,)
0.002743244171142578
(6,)
(10,)
(13,)
(3,)
(7,)
(4,)
0.005016326904296875
(6,)
(3,)
(7,)
(8,)
0.002136707305908203
(12,)
(12,)
0.0017218589782714844
(14,)
(13,)
(12,)
(14,)
(3,)
(4,)
(8,)
0.005075931549072266
(14,)
(12,)
(3,)
(8,)
0.003960847854614258
(6,)
(14,)
(13,)
(3,)
0.003656148910522461
(6,)
(13,)
(3,)
(4,)
(2,)
(3,)
(7,)
0.0043239593505859375
(13,)
(12,)
(4,)
0.0023844242095947266
(10,)
(16,)
(3,)
0.001985311508178711
(3,)
(2,)
(8,)
(7,)
(18,)
0.0028944015502929688
(14,)
(12,)
(12,)
(13,)
(14,)
(14,

(14,)
(12,)
(13,)
(14,)
(3,)
(16,)
0.004675865173339844
(12,)
(12,)
(13,)
(14,)
(3,)
(7,)
(8,)
0.003520965576171875
(12,)
(13,)
(16,)
0.003513336181640625
(12,)
(7,)
(12,)
(3,)
(8,)
0.0031905174255371094
(6,)
(3,)
(5,)
(7,)
(3,)
(12,)
(7,)
(7,)
(8,)
(8,)
0.004936695098876953
(6,)
(12,)
(13,)
(12,)
(12,)
(4,)
(14,)
(8,)
0.004564523696899414
(6,)
(7,)
(3,)
(2,)
(8,)
0.002104520797729492
(14,)
(13,)
(12,)
(7,)
(3,)
(12,)
(7,)
(7,)
0.004771709442138672
(14,)
(14,)
(13,)
(5,)
(3,)
(12,)
(14,)
0.004569530487060547
(3,)
(12,)
(3,)
(8,)
(8,)
0.0022361278533935547
(6,)
(12,)
(13,)
(8,)
(8,)
0.003263711929321289
(6,)
(13,)
(12,)
(7,)
(12,)
(3,)
(2,)
(8,)
(8,)
0.005388975143432617
(6,)
(6,)
(13,)
(14,)
(7,)
(12,)
(14,)
0.004300355911254883
(6,)
(7,)
(13,)
(14,)
(3,)
0.0031065940856933594
(6,)
(3,)
(12,)
(3,)
(12,)
(7,)
0.0035521984100341797
(6,)
(3,)
(3,)
(3,)
(7,)
(2,)
(7,)
(7,)
(8,)
0.003986835479736328
(6,)
(12,)
(7,)
(3,)
0.0035948753356933594
(12,)
(14,)
(8,)
(3,)
(8,)
0.003013134002685547
(

In [61]:
print(f"average latency: {int(np.mean(lat)*100000)/100}ms")
pred = np.asarray(res)
pred[pred<=1.0] = 1
cards = np.asarray(cards)
errors = np.maximum(np.divide(pred, cards), np.divide(pred, cards))
for i in [50,90,95,99,100]:
    print(f"{i}% quantile: {np.percentile(errors, i)}")

average latency: 3.48ms
50% quantile: 1.003072607689412
90% quantile: 1.3808882874589132
95% quantile: 1.8733718240353872
99% quantile: 5.566838830940414
100% quantile: 115.27912982882839


In [ ]:
len(pred)

In [62]:
import torch

In [ ]:
a = torch.ones((3,4))

In [ ]:
b = torch.tensor([1.0,2.0,3.0])

In [ ]:
torch.ones(torch.matmul(b, a).shape)

In [ ]:
torch.ones(a.shape[-1])

In [ ]:
torch.prod(torch.stack([b, torch.ones(3), b]), 0)

In [ ]:
torch.matmul(a[:,1], b).item()